# **Human-Detection-Using-Machine-Learning**

## Setup Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/data-set.zip -d /content/

In [ ]:
!pip install ultralytics

In [ ]:
import yaml

# Define the content you want to write to the YAML file
data = {
    'path': '/content/dataset/data.yaml',
    'train': '/content/dataset/train/images',
    'val': '/content/dataset/valid/images',
    'test': '/content/dataset/test/images',
    'names': {
        0: 'person'
    }
}

# Path to your YAML file
yaml_file_path = '/content/dataset/data.yaml'

# Write the content to the YAML file
with open(yaml_file_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"YAML file '{yaml_file_path}' has been overwritten successfully.")

## Train the model using YOLO

In [ ]:
from ultralytics import YOLO # Load the YOLOv8 model
model = YOLO('yolo11x-obb.pt')  # Choose a model variant
# Train the model using the data.yaml configuration
model.train(data='/content/dataset/data.yaml', epochs=15, imgsz=640)

In [ ]:
from ultralytics import YOLO # Load the YOLOv8 model
model = YOLO('yolo11x-obb.pt')  # Choose a model variant
dataset_path = "/content/dataset/data.yaml" # Choose the Dataset Path
# Train the model using the data.yaml configuration
train_results = model.train(
    data=dataset_path,
    epochs=50,          # More epochs for better accuracy
    imgsz=640,         # Larger image size improves detection
    device="cuda",      # Use GPU in Colab
    workers=8,         # More workers for faster data loading
    batch=16,          # Larger batch size for GPU efficiency
    augment=True,      # Enable data augmentation
    lr0=0.0005,        # Lower learning rate for stability
    patience=10,       # Early stopping if no improvement
    optimizer="AdamW", # AdamW optimizer for better training stability
    save=True,         # Save the best model
    project="/content/yolo_training_results",  # Save outputs to Google Drive
)

## Test trained model

### Image Test

In [ ]:
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import cv2
import torch

# Load the YOLOv8 model
model = YOLO('/content/yolo_training_resultsruns/detect/train/weights/best.pt')

# Load a test image
image_path = '/content/dataset/26-08-2024/test/images/fsi122_jpg.rf.522dadebf50d11555d7317bd3f870330.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Perform inference
results = model(image)

# Process and display the results
for result in results:
    boxes = result.boxes
    for box in boxes:
        # Get the coordinates of the bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

        # Draw the bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Get the label and confidence score
        label = model.names[int(box.cls)]
        confidence = box.conf[0]

        # Display label and confidence on the image
        cv2.putText(image, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the result
cv2.imwrite('output/output.jpg', image)

### Video Test

In [ ]:
from ultralytics import YOLO
# Load the trained YOLO model
model = YOLO("/content/runs/detect/train/weights/best.pt")
# Run inference on the video
results = model.predict(source="Input/Vid/Input.mp4", save=True, save_crop=True)

## Output Rendering of Video's

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Paths
model_path = r'/content/runs/detect/train/weights/best.pt'
output_folder = r'/content/output'
output_video_path = os.path.join(output_folder, 'annotated_output.mp4')

# Create output folder if it doesn't exist
if not os.path.exists(output_folde):
    os.makedirs(output_folder)

# Load the trained model
model = YOLO(model_path)

# For recorded video input:
video_source = r'/content/Video.mp4'

# Open video capture
cap = cv2.VideoCapture(video_source)

# Check if the video source opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video source {video_source}")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_number = 0
while True:
    ret, frame = cap.read()

    if not ret:
        print("End of video stream or failed to capture frame.")
        break

    # Perform inference
    results = model.predict(source=frame)

    # Process results
    annotated_frame = False  # Flag to track if any person was detected

    for result in results:
        boxes = result.boxes
        names = result.names

        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()  # Get bounding box coordinates
                cls = int(box.cls[0])  # Get class index
                conf = box.conf[0]  # Get confidence score
                label = names[cls]

                if label == 'person' and conf >= 0.3:  # Check if detected object is a person with a confidence score above a threshold
                    annotated_frame = True  # Set the flag to True if a person is detected

                    # Draw bounding boxes and labels
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f'{label} {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    if annotated_frame:
        # Write the annotated frame to the video file
        out.write(frame)

    # Display the frame (optional)
    # cv2.imshow('YOLO Detection', frame)

    # Press 'q' to quit the video loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_number += 1

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Annotated video saved at: {output_video_path}')

## Downlord the output

In [ ]:
import zipfile
import os

def zip_directory(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# Path to the folder you want to zip
folder_to_zip = '/content/output'
# Path where you want to save the zip file
zip_file_path = '/content/results.zip'

zip_directory(folder_to_zip, zip_file_path)